In [26]:
import json
import pandas as pd
import geopandas as gpd
import altair as alt

import vega_datasets
from ast import literal_eval

import numpy as np
import folium
from folium.plugins import MeasureControl

In [27]:
def array_eval(x):
    array_str = ','.join(x.split())
    array_str = array_str.replace('[,', '[')
    array_str = array_str.replace('\n', '')
    return np.array(literal_eval(array_str))

gdf = gpd.read_file(r"C:\BERATools\Test-data\footprint-fixed-avg.geojson")    
gdf['sampling_widths'] = gdf['sampling_widths'].apply(array_eval)

In [28]:
def add_histgram(widths):
    width_max, width_min = max(widths), min(widths)
    widths_pd = pd.DataFrame({'sampling_widths': widths})
    width_max, width_min

    hist = alt.Chart(widths_pd).mark_bar().encode(
    alt.X("sampling_widths", bin=alt.Bin(extent=[width_min, width_max], step=1.0)),
    y='count()',
    )

    vega_lite = folium.VegaLite(
        hist,
        width="100%",
        height="100%",
    )

    return vega_lite


In [29]:
# Create two maps.
m = folium.Map(
    location=[56.201831,-110.672306],
    zoom_start=14,
    position="absolute",
    left="0%"
)

m.add_child(MeasureControl())

for index, row in gdf.iterrows(): 
    widths = row['sampling_widths']
    polygon = folium.GeoJson(row['geometry'])

    popup = folium.Popup()
    vega_lite = add_histgram(widths)
    vega_lite.add_to(popup)
    popup.add_to(polygon)
    polygon.add_to(m)

In [30]:
m